# Topic Modelling using BERTopic & cuBERTopic

Sample notebook to show cuBERTopic, a topic modelling technique that is built on top of the NVIDIA RAPIDS ecoysystem, utilizing libraries such as `cudf` and `cuml` to GPU-accelarate end-to-end workflow for extracting topic from a set of documents. We run the same operations using `BERTopic` to compare their behaviour. 

## Quick Start
In both the cases, we start by extracting topics from the well-known 20 newsgroups dataset from `sklearn` which is comprised of english documents

### Installing relevant packages
Here we need to install relevant dependencies for `BERTopic` as well as we compare performance between it and `cuBERTopic`. 

`cuBERTopic` runs on `cudf` and `cuml` which can be installed using instructions at https://rapids.ai/start.html and `cupy` which can be installed from https://docs.cupy.dev/en/stable/install.html

More detailed instructions are in the README.

In [1]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from transformers import AutoTokenizer, AutoModel
import torch
from cuBERTopic import gpu_BERTopic

docs = fetch_20newsgroups(subset='all')['data']

### Running `BERTopic`
`BERTopic` provides us the functionality of providing custom embeddings, so we create sentence embeddings using `AutoTokenizer` followed by `AutoModel` from `transformers` and pass it to `fit_transform` method inside `BERTopic` class, which fits the models on a collection of documents, generate topics, and return the docs with topics.

In [2]:
%%time
# Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2"
)

# Tokenize sentences
encoded_input = tokenizer(
    docs,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask
        .unsqueeze(-1)
        .expand(token_embeddings.size())
        .float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(
    model_output,
    encoded_input["attention_mask"]
)
sentence_embeddings = sentence_embeddings.to('cpu').numpy()
topic_model = BERTopic()
topics_cpu, probs_cpu = topic_model.fit_transform(docs, sentence_embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

`get_topic_info` returns information about each topic including its id, frequency, and name 

In [3]:
%%time
topic_model.get_topic_info()

CPU times: user 3.24 ms, sys: 0 ns, total: 3.24 ms
Wall time: 2.92 ms


,Topic,Count,Name
0,-1,6054,-1_drive_system_email_information
1,0,502,0_monitor_card_video_drivers
2,1,371,1_gun_guns_firearms_weapons
3,2,184,2_car_cars_toyota_automotive
4,3,169,3_printer_deskjet_print_hp
...,...,...,...
382,381,10,381_cd_oversampling_player_filter
383,382,10,382_hockey_hispanic_ice_sport
384,383,10,383_clipper_phone_keys_conversation
385,384,10,384_winfield_peak_winfields_lurie


`get_topic` returns topics with top n words and their c-TF-IDF score

-1 refers to all outliers and should typically be ignored. Next, let's take a look at the most frequent topic that was generated, topic 0:

In [4]:
%%time
topic_model.get_topic(0)

CPU times: user 10 µs, sys: 4 µs, total: 14 µs
Wall time: 22.9 µs


[('monitor', 0.01335695430028301),
 ('card', 0.012357331665604408),
 ('video', 0.011406622681194576),
 ('drivers', 0.008313304113936508),
 ('vga', 0.007676716939376939),
 ('monitors', 0.006883837547712011),
 ('diamond', 0.006440620960961699),
 ('ati', 0.006397267237023263),
 ('screen', 0.005291875700017211),
 ('driver', 0.0052012420367266166)]

### Running `cuBERTopic`
`cuBERTopic` provides with a similar API for passing in `docs` as a set of strings to model on. Here, instead of using `AutoTokenizer` from `transformers`, we use `SubwordTokenizer` from `cuDF` in combination with `AutoModel` from `transformers`. 

Due to the stochastisch nature of UMAP, the results might differ and the quality can degrade.

In [5]:
%%time
gpu_topic = gpu_BERTopic()
topics_gpu, probs_gpu = gpu_topic.fit_transform(docs)

Label prop iterations: 22
Label prop iterations: 8
Label prop iterations: 6
Label prop iterations: 4
Label prop iterations: 6
Iterations: 5
2697,159,329,16,284,1472
Label prop iterations: 4
Label prop iterations: 2
Iterations: 2
1272,72,156,6,105,224
Label prop iterations: 2
Iterations: 1
1003,48,104,11,59,90
CPU times: user 30.6 s, sys: 5.75 s, total: 36.4 s
Wall time: 28.8 s


In [6]:
%%time
gpu_topic.get_topic_info()

CPU times: user 16.3 ms, sys: 2.5 ms, total: 18.8 ms
Wall time: 17.1 ms


,Topic,Count,Name
0,-1,6697,-1_email_anyone_information_god
50,0,358,0_gun_guns_firearms_weapons
271,1,182,1_hockey_leafs_team_nhl
221,2,148,2_bike_bikes_honda_motorcycle
196,3,136,3_morality_objective_moral_values
...,...,...,...
339,390,10,390_smiley_object_kuiper_karla
344,391,10,391_tapes_contains_bray_duran
347,392,10,392_vlb_card_orchid_vga
380,393,10,393_moon_luna_coffman_lunar


In [7]:
%%time
gpu_topic.get_topic(0)

CPU times: user 124 ms, sys: 7.9 ms, total: 132 ms
Wall time: 131 ms


[('gun', array(0.01540629)),
 ('guns', array(0.00905086)),
 ('firearms', array(0.00663373)),
 ('weapons', array(0.00574954)),
 ('militia', array(0.00550295)),
 ('weapon', array(0.00457949)),
 ('control', array(0.00456093)),
 ('amendment', array(0.00455747)),
 ('handgun', array(0.00446296)),
 ('firearm', array(0.0043654))]